In [1]:
from collections import deque
from copy import deepcopy
from itertools import combinations, permutations

# Part 1

In [2]:
test = ['Valve AA has flow rate=0; tunnels lead to valves DD, II, BB',
        'Valve BB has flow rate=13; tunnels lead to valves CC, AA',
        'Valve CC has flow rate=2; tunnels lead to valves DD, BB',
        'Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE',
        'Valve EE has flow rate=3; tunnels lead to valves FF, DD',
        'Valve FF has flow rate=0; tunnels lead to valves EE, GG',
        'Valve GG has flow rate=0; tunnels lead to valves FF, HH',
        'Valve HH has flow rate=22; tunnel leads to valve GG',
        'Valve II has flow rate=0; tunnels lead to valves AA, JJ',
        'Valve JJ has flow rate=21; tunnel leads to valve II']

In [3]:
def generate_graph(data):
    graph = {}
    for line in data:
        line = line.strip().split()
        name = line[1]
        flow_rate = int(line[4][5:-1])
        connections = []
        for i in range(9, len(line)):
            connections.append(line[i][:2])
        graph[name] = [flow_rate, connections]
    return graph

#Breadth-First Search
def BFS(graph, start):
    queue = deque([start])
    dist = {start: 0}
    tgt_dist = {}
    
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist = dist[cur_pos]
        nxt_dist = cur_dist + 1
        
        for nxt_pos in graph[cur_pos][1]:
            if nxt_pos not in dist.keys():
                queue.append(nxt_pos)
                dist[nxt_pos] = nxt_dist
            if graph[nxt_pos][0] > 0 and nxt_pos not in tgt_dist.keys():
                tgt_dist[nxt_pos] = nxt_dist
                
    return tgt_dist

def reduce_graph(graph):
    start = 'AA'
    new_graph = {}
    new_graph[start] = [graph[start][0], BFS(graph, start)]
    valves = new_graph[start][1].keys()
    for valve in valves:
        new_graph[valve] = [graph[valve][0], BFS(graph, valve)]
    return new_graph
    
def DFS(graph, start, discovered, dt, t, flow_rate, flow, all_flows):
    discovered.append(start)
    
    if start != 'AA':
        if t+dt+1 > 30:
            flow += (30-t)*flow_rate
            all_flows.append(flow)
            return
        flow += (dt+1)*flow_rate
        t += dt+1
        flow_rate += graph[start][0]
    
    count = len(list(graph[start][1].keys()))
    for nxt_pos in list(graph[start][1].keys()):
        if nxt_pos not in discovered:
            DFS(graph, nxt_pos, deepcopy(discovered), 
                graph[start][1][nxt_pos], deepcopy(t), 
                deepcopy(flow_rate), deepcopy(flow), all_flows)
        else:
            count -= 1
            
    if count == 0:
        flow += (30-t)*flow_rate
        all_flows.append(flow)
    return

def run_part1(data):
    graph = generate_graph(data)
    graph = reduce_graph(graph)
    all_flows = []
    DFS(graph, 'AA', [], 0, 0, 0, 0, all_flows)
    print('Part 1 result:', max(all_flows))

In [4]:
run_part1(test)

Part 1 result: 1651


In [5]:
with open('day16_input.txt', 'r') as f:
    inpt = f.readlines()
    f.close()
run_part1(inpt)

Part 1 result: 2265


# Part 2

In [6]:
def DFS2(graph, start, discovered, dt, t, flow_rate, flow, all_flows, path, future):
    discovered.append(start)
    path.append(start)
    
    if start != 'AA':
        if t+dt+1 >= 26:
            flow += (26-t)*flow_rate
            all_flows.append(flow)
            return
        flow += (dt+1)*flow_rate
        t += dt+1
        flow_rate += graph[start][0]
        
        if (tuple(path[1:]) in future.keys() and flow + ((26-t)*flow_rate) > future[tuple(path[1:])]):
            future[tuple(path[1:])] = flow + ((26-t)*flow_rate)
        elif tuple(path[1:]) not in future.keys():
            future[tuple(path[1:])] = flow + ((26-t)*flow_rate)
    
    count = len(list(graph[start][1].keys()))
    for nxt_pos in list(graph[start][1].keys()):
        if nxt_pos not in discovered:
            DFS2(graph, nxt_pos, deepcopy(discovered), 
                 graph[start][1][nxt_pos], deepcopy(t), 
                 deepcopy(flow_rate), deepcopy(flow), all_flows,
                 deepcopy(path), future)
        else:
            count -= 1
            
    if count == 0:
        flow += (26-t)*flow_rate
        all_flows.append(flow)
    return

def two_player(graph, future, limit):
    flow = 0
    nodes = list(graph.keys())
    nodes.remove('AA')
    
    key_list = list(future.keys())
    for key in key_list:
        if len(key) == len(nodes):
            del future[key]
            continue
    
    key_list = sorted(future, key=future.get, reverse=True)
    if len(key_list) > 2000:
        key_list = key_list[:2000]
                
    for max_key in key_list:
        maxi = future[max_key]
        
        other_nodes = deepcopy(nodes)
        for c in max_key:
            other_nodes.remove(c)
        for i in range(1, len(other_nodes)+1):
            for co in combinations(other_nodes, i):
                if co not in future.keys():
                    continue
                if maxi+future[co] > flow:
                    flow = maxi+future[co]
                
    return flow
        
def run_part2(data):
    graph = generate_graph(data)
    graph = reduce_graph(graph)
    all_flows = []
    future = {}
    DFS2(graph, 'AA', [], 0, 0, 0, 0, all_flows, [], future)   
    print('Part 2 result:', two_player(graph, future, max(all_flows))) 

In [7]:
run_part2(test)

Part 2 result: 1706


In [8]:
run_part2(inpt)

Part 2 result: 2811
